In [ ]:
# app.py

# Importiere benötigte Bibliotheken für die Webanwendung und Datenverarbeitung [1]
from flask import Flask, request, jsonify, render_template
import pandas as pd

# Initialisiere die Flask-Anwendung [1]
app = Flask(__name__)

# Eine einfache In-Memory-Datenbank (Pandas DataFrame) zur Speicherung der Daten [1]
# Diese Datenbank wird bei jedem Neustart der Anwendung zurückgesetzt.
database = pd.DataFrame()

# =================================================================
# WEB-INTERFACE-ROUTEN
# =================================================================

# Route für die Startseite des Web-Interfaces [2]
# Zeigt eine HTML-Seite zum Hochladen von CSV-Dateien und die bereits hochgeladenen Daten an.
@app.route('/')
def index():
    global database # Greife auf die globale 'database'-Variable zu

    # Konvertiere den DataFrame in eine HTML-Tabelle, wenn Daten vorhanden sind [2]
    # Ansonsten wird 'No data available.' angezeigt.
    table_html = database.to_html(index=False) if not database.empty else "No data available."

    # Rendere die 'index.html'-Vorlage und übergebe die HTML-Tabelle [2]
    return render_template('index.html', table=table_html)

# Route zum Hochladen von CSV-Dateien über das Web-Interface [2]
@app.route('/upload', methods=['POST'])
def upload_data_web():
    global database # Greife auf die globale 'database'-Variable zu

    # Überprüfe, ob eine Datei im Request enthalten ist [2]
    if 'file' not in request.files:
        return "No file provided", 400 # Fehlermeldung, wenn keine Datei gefunden wird

    file = request.files['file'] # Hole die hochgeladene Datei [2]

    # Überprüfe, ob die Dateiendung '.csv' ist [2]
    if not file.filename.endswith('.csv'):
        return "Only CSV files are allowed", 400 # Fehlermeldung, wenn es keine CSV-Datei ist

    try:
        # Lese die CSV-Daten mit Pandas ein [3]
        data = pd.read_csv(file)

        # Hänge die neuen Daten an die vorhandene Datenbank an [3]
        # 'ignore_index=True' stellt sicher, dass der Index neu generiert wird.
        database = pd.concat([database, data], ignore_index=True)

        # Erfolgsmeldung und Statuscode 201 (Created) [3]
        return "Data uploaded successfully. Go back <a href='/'>Home</a>", 201
    except Exception as e:
        # Fehlermeldung bei Problemen mit der CSV-Verarbeitung [3]
        return f"Failed to process CSV: {str(e)}. Go back <a href='/'>Home</a>", 400

# =================================================================
# RESTFUL-API-ROUTEN
# =================================================================

# API-Endpunkt zum Abrufen aller gespeicherten Daten im JSON-Format [3]
@app.route('/api/daten', methods=['GET'])
def get_data():
    global database # Greife auf die globale 'database'-Variable zu

    # Überprüfe, ob die Datenbank leer ist [3]
    if database.empty:
        # Wenn leer, sende eine Nachricht und Statuscode 404 (Not Found) [3]
        return jsonify({'message': 'No data available'}), 404

    # Konvertiere den DataFrame in JSON und gib ihn zurück [3]
    # 'orient='records'' formatiert die Daten als Liste von JSON-Objekten (Zeilen).
    return database.to_json(orient='records')

# Startet den Flask-Entwicklungsserver, wenn das Skript direkt ausgeführt wird [4]
# debug=True: Aktiviert den Debug-Modus (gut für die Entwicklung).
# use_reloader=False: Deaktiviert den Reloader, um Probleme mit doppelten Ausführungen zu vermeiden.
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

In [ ]:
# analyze_data.py

# Importiere benötigte Bibliotheken
import requests  # Zum Abrufen von Daten von der API
import pandas as pd  # Für die Datenverarbeitung mit DataFrames
from sklearn.model_selection import train_test_split  # Zum Aufteilen der Daten in Trainings- und Testsets
from sklearn.linear_model import LinearRegression  # Für das lineare Regressionsmodell
from sklearn.metrics import mean_squared_error  # Zur Bewertung des Modells (Mittlerer Quadratischer Fehler)
import sys # Für sys.exit()

print("Starte Datenabruf und Analyse für Tag 2...\n")

# =================================================================
# 1. DATEN VON DER API ABRUFEN
# =================================================================
# Definiere die URL des API-Endpunkts
API_URL = "http://127.0.0.1:5000/api/daten"
data = None  # Initialisiere data mit None

try:
    print(f"Versuche, Daten von API abzurufen: {API_URL}")
    response = requests.get(API_URL, timeout=10)  # Timeout hinzugefügt
    response.raise_for_status()  # Löst eine HTTPError-Exception für 4XX/5XX Statuscodes aus

    # Konvertiere die JSON-Antwort in einen Pandas DataFrame
    # Es wird erwartet, dass die API eine Liste von Dictionaries zurückgibt
    json_data = response.json()
    if isinstance(json_data, list) and json_data: # Prüfen, ob es eine nicht-leere Liste ist
        data = pd.DataFrame(json_data)
        print("Daten erfolgreich von der API abgerufen:")
        print(data.head())
        print(f"\nAnzahl der abgerufenen Datensätze: {len(data)}")
    elif isinstance(json_data, dict) and json_data.get('message') == 'No data available':
        print("API meldet: Keine Daten verfügbar.")
        data = pd.DataFrame() # Leeres DataFrame, wenn API keine Daten hat
    elif not json_data:
        print("API hat eine leere Datenliste zurückgegeben.")
        data = pd.DataFrame()
    else:
        print(f"Unerwartetes Datenformat von der API erhalten: {type(json_data)}")
        data = pd.DataFrame()

except requests.exceptions.Timeout:
    print(f"Fehler: Timeout beim Versuch, die API unter {API_URL} zu erreichen.")
    sys.exit("Skript wird beendet. Bitte API-Verfügbarkeit prüfen.")
except requests.exceptions.ConnectionError:
    print(f"Fehler: Verbindung zur API unter {API_URL} fehlgeschlagen.")
    sys.exit("Stellen Sie sicher, dass die Flask-Anwendung (app.py) läuft. Skript wird beendet.")
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP-Fehler beim Abrufen der Daten: {http_err}")
    sys.exit("Skript wird beendet.")
except requests.exceptions.JSONDecodeError:
    print("Fehler: Die Antwort der API konnte nicht als JSON dekodiert werden.")
    sys.exit("Skript wird beendet.")
except Exception as e:
    print(f"Ein unerwarteter Fehler ist beim API-Abruf aufgetreten: {e}")
    sys.exit("Skript wird beendet.")

# Überprüfen, ob Daten erfolgreich geladen wurden
if data is None or data.empty:
    print("\nKeine Daten zum Verarbeiten vorhanden (entweder von API oder API war leer).")
    sys.exit("Skript wird beendet, da keine Daten für die Analyse vorliegen.")

print("\nSpalten im DataFrame vor der Vorverarbeitung:", data.columns.tolist())

# =================================================================
# 2. DATENVORVERARBEITUNG
# =================================================================

print("\nStarte Datenvorverarbeitung...")

# Definiere Feature- und Zielspaltennamen für bessere Wartbarkeit
FEATURE_COLUMNS = ['Grundstücksgröße', 'Zimmeranzahl', 'Garagenanzahl']
TARGET_COLUMN = 'Hauspreis'
ALL_RELEVANT_COLUMNS = FEATURE_COLUMNS + [TARGET_COLUMN]

# Überprüfen, ob alle relevanten Spalten vorhanden sind
missing_cols = [col for col in ALL_RELEVANT_COLUMNS if col not in data.columns]
if missing_cols:
    print(f"Fehler: Folgende benötigte Spalten fehlen im DataFrame: {missing_cols}")
    print(f"Verfügbare Spalten sind: {data.columns.tolist()}")
    sys.exit("Skript wird beendet. Bitte Datenquelle überprüfen.")

# Datentypkonvertierung und Umgang mit Fehlern
for col in ALL_RELEVANT_COLUMNS:
    # Versuche, in numerische Werte umzuwandeln, Fehler werden zu NaN
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Fehlende Werte entfernen (nach der Konvertierung)
# Dies entfernt Zeilen, wenn in einer der relevanten Spalten ein NaN-Wert steht
data_cleaned = data.dropna(subset=ALL_RELEVANT_COLUMNS)
print(f"Anzahl der Datensätze vor dem Entfernen fehlender Werte: {len(data)}")
print(f"Anzahl der Datensätze nach dem Entfernen fehlender Werte: {len(data_cleaned)}")

# Überprüfe, ob nach dem Entfernen fehlender Werte noch Daten übrig sind
if data_cleaned.empty:
    print("Keine ausreichenden Daten für die Analyse nach dem Entfernen fehlender Werte vorhanden.")
    sys.exit("Bitte überprüfen Sie Ihre Datenquelle und die Konvertierung.")

# Definiere Features (unabhängige Variablen) und Zielvariable (abhängige Variable)
X = data_cleaned[FEATURE_COLUMNS]
y = data_cleaned[TARGET_COLUMN]

print("Features (X) und Zielvariable (y) definiert.")
print("Beispiel Features (X.head()):\n", X.head())
print("Beispiel Zielvariable (y.head()):\n", y.head())

# =================================================================
# 3. TRAININGS- UND TESTDATEN SPLITTEN
# =================================================================

print("\nSplitting der Daten in Trainings- und Testsets...")
if len(X) < 2: # Benötigt mindestens 2 Samples für train_test_split
    print("Nicht genügend Daten für das Splitten in Trainings- und Testsets vorhanden.")
    sys.exit("Skript wird beendet.")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Trainingsdaten-Größe (X_train): {X_train.shape}")
print(f"Testdaten-Größe (X_test): {X_test.shape}")

# =================================================================
# 4. LINEARE REGRESSION TRAINIEREN
# =================================================================

print("\nTrainiere Lineares Regressionsmodell...")
model = LinearRegression()
model.fit(X_train, y_train)

print("Modell erfolgreich trainiert.")
print(f"Modellkoeffizienten (Steigungen): {model.coef_}")
print(f"Modell-Achsenabschnitt (Intercept): {model.intercept_}")

# =================================================================
# 5. VORHERSAGE
# =================================================================

print("\nFühre Vorhersagen auf den Testdaten durch...")
y_pred = model.predict(X_test)
print("Vorhersagen erstellt.")

# =================================================================
# 6. MODELLBEWERTUNG
# =================================================================

print("\nBewerte das Modell...")
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5 # Root Mean Squared Error ist oft interpretierbarer
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

# =================================================================
# 7. BEISPIELVORHERSAGE
# =================================================================

print("\nFühre Beispielvorhersagen für neue Daten durch...")
sample_data_dict = {
    'Grundstücksgröße': [150, 200, 120, 250, 180],
    'Zimmeranzahl': [3, 4, 2, 5, 3],
    'Garagenanzahl': [1, 2, 1, 2, 0]
}
sample_df = pd.DataFrame(sample_data_dict)

# Stelle sicher, dass die Spalten in der gleichen Reihenfolge wie beim Training sind
sample_df_ordered = sample_df[FEATURE_COLUMNS]

predicted_prices = model.predict(sample_df_ordered)

print("Geschätzte Hauspreise für Beispieldaten:")
for i, price in enumerate(predicted_prices):
    print(f"Beispiel {i+1}: Grundstücksgröße={sample_df_ordered.loc[i, 'Grundstücksgröße']}, "
          f"Zimmeranzahl={sample_df_ordered.loc[i, 'Zimmeranzahl']}, "
          f"Garagenanzahl={sample_df_ordered.loc[i, 'Garagenanzahl']} -> "
          f"Geschätzter Hauspreis: {price:.2f} €")

print("\nAnalyse für Tag 2 abgeschlossen.")
